In [1]:
pip install scikit-learn pandas numpy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
df = pd.read_json("recipes.json")
df

,id,title,content,ingredients
0,0,Abált szalonna,Frissen bontott tokaszalonnát sós vízben forrá...,"[tokaszalonna, só, fokhagyma, fűszerpaprika, f..."
1,1,Áfonya torta,A makadámdiót ledaráljuk. A tésztához a szójat...,"[tönköly búzaliszt, makadámdió, szójatejföl, n..."
2,2,Áfonyás húsgolyó,25 dkg kukoricadarát annyi sós vízben megfőzün...,[]
3,3,"Áfonyás, kevert sütemény mandulával",A sütőt 190 fokra előmelegítjük. Egy 20x30 cm-...,"[a morzsához:, 40 g liszt, 100 g cukor, 1 tk ő..."
4,4,Áfonyás-narancsos sajttorta,"A kekszet, a zabpelyhet és a nádcukrot összeda...","[5 dk teljes kiőrlésű búzadarás keksz, 5 dkg z..."
...,...,...,...,...
4917,4917,Zöldségleves csipetkével és csirkével,"A zöldségeket megmosom, kockára vágom. A hagym...","[1/2 csirke,, 1 kg burgonya,, 3 új sárgarépa,,..."
4918,4918,Zsidó tojás,A hagymát apróra vágjuk és serpenyőben megpirí...,"[0 .5 kg kacsamáj, 2 db tojás, mustár, 1 nagy ..."
4919,4919,Zöld almás görögdinnye frissítő,Egyszerű dolgunk van. A kimagozott görögdinnye...,"[7,5 dl görögdinnyevelő, 2 db zöld alma, 20 sz..."
4920,4920,Zserbó,A tésztához felfuttatjuk az élesztőt. A liszte...,"[50 dkg liszt, 10 dkg cukor, 20 dkg zsír, 1,5 ..."


In [4]:
df['ingredients_str'] = df['ingredients'].apply(lambda x: " ".join(x))

# Check the preprocessed data
print(data[['title', 'ingredients_str']].head())

In [ ]:
vectorizer = TfidfVectorizer()
ingredient_vectors = vectorizer.fit_transform(df['ingredients_str'])

# Find similar recipes
similarity_matrix = cosine_similarity(ingredient_vectors)

In [5]:
def recommend_recipe(recipe_id, similarity_matrix, df, top_n=5):
    sim_scores = list(enumerate(similarity_matrix[recipe_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_recipes = [df.iloc[i[0]] for i in sim_scores[1:top_n+1]]
    return top_recipes

In [6]:
recommend_recipe(3, similarity_matrix, df)

[id                                                              4590
 title                            Áfonyás, kevert sütemény mandulával
 content            A sütőt 190 fokra előmelegítjük. Egy 20x30 cm-...
 ingredients        [a morzsához:, 40 g liszt, 100 g cukor, 1 tk ő...
 ingredients_str    a morzsához: 40 g liszt 100 g cukor 1 tk őrölt...
 Name: 4590, dtype: object,
 id                                                              3588
 title                                            Céklás-almás morzsa
 content            Egy keverőtálban összekeverünk mandulalisztet,...
 ingredients        [50 g pekándió, 50 g fenyőmag, 150 g cékla, ba...
 ingredients_str    50 g pekándió 50 g fenyőmag 150 g cékla balzsa...
 Name: 3588, dtype: object,
 id                                                               703
 title                                                         Éclair
 content            A sütőt 200 fokra előmelegítjük A tejet, vizet...
 ingredients        [a tésztához:,

In [9]:
# Normalize ingredients (convert to lowercase for consistency)
df['ingredients_normalized'] = df['ingredients'].apply(
    lambda x: [ingredient.lower() for ingredient in x]
)

# Check the normalized data
print(df[['title', 'ingredients_normalized']].head())

                                 title  \
0                       Abált szalonna   
1                         Áfonya torta   
2                     Áfonyás húsgolyó   
3  Áfonyás, kevert sütemény mandulával   
4          Áfonyás-narancsos sajttorta   

                              ingredients_normalized  
0  [tokaszalonna, só, fokhagyma, fűszerpaprika, f...  
1  [tönköly búzaliszt, makadámdió, szójatejföl, n...  
2                                                 []  
3  [a morzsához:, 40 g liszt, 100 g cukor, 1 tk ő...  
4  [5 dk teljes kiőrlésű búzadarás keksz, 5 dkg z...  


In [10]:
def find_recipes_by_ingredients(user_ingredients, data, top_n=5):
    user_ingredients = [ingredient.lower() for ingredient in user_ingredients]  # Normalize user input
    
    # Calculate overlap score for each recipe
    data['ingredient_overlap'] = data['ingredients_normalized'].apply(
        lambda recipe_ingredients: len(set(user_ingredients) & set(recipe_ingredients))
    )
    
    # Sort recipes by overlap score in descending order
    recommendations = data.sort_values(by='ingredient_overlap', ascending=False).head(top_n)
    
    # Return the top matching recipes
    return recommendations[['id', 'title', 'ingredient_overlap']]

# Example usage
user_ingredients = ["vaj", "fokhagyma", "liszt"]  # User input
print(find_recipes_by_ingredients(user_ingredients, df))

        id                                              title  \
2409  2409                       Radicchios-mentás palacsinta   
2338  2338                            Pisztráng májjal töltve   
2831  2831                                  Sütőtökös rizottó   
2627  2627         Scampikrémmel töltött tintahal brokkolival   
328    328  Borjútekercs marsalaval ízesített kacsamájjal ...   

      ingredient_overlap  
2409                   3  
2338                   3  
2831                   3  
2627                   3  
328                    3  


In [15]:
def combined_recommendation(user_ingredients, similarity_matrix, data, top_n=5):
    # Match by ingredients
    ingredient_matches = find_recipes_by_ingredients(user_ingredients, data, top_n=None)
    
    # Add similarity scores
    ingredient_matches['similarity_score'] = ingredient_matches['id'].apply(
        lambda recipe_id: similarity_matrix[data.index[data['id'] == recipe_id][0]].mean()
    )
    
    # Combine scores (weighted average)
    ingredient_matches['combined_score'] = ingredient_matches['ingredient_overlap'] * 0.7 + ingredient_matches['similarity_score'] * 0.3
    
    # Sort by combined score
    recommendations = ingredient_matches.sort_values(by='combined_score', ascending=False).head(top_n)
    
    return recommendations[['id', 'title', 'combined_score']]

# Example usage
user_ingredients = ["alma", "körte", "cseressznye"]  # User input
print(combined_recommendation(user_ingredients, similarity_matrix, df))


        id                                              title  combined_score
2800  2800  Sült libacomb diós nudlival és sült gyümölcsökkel        1.410270
34      34                                       Almás kalács        0.716719
3622  3622  Csillagánizsos körte-sütőtök kompót - Segal Vi...        0.716102
4393  4393                 Édes, savanyú hagymaszószos kagyló        0.711262
3518  3518                                   Alma pongyolában        0.709784
